# Input names

In [110]:
newSceneName = "SceneName"
userHistoryEventSceneStart = f"Перешел к сцене {newSceneName}"
tab = '    '

# Generation

In [111]:
import re

def toCamelCaseFromSnake(snake_str):
    return "".join(x.capitalize() for x in snake_str.lower().split("_"))

def toSnakeCaseFromCamel(name):
    name = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    return re.sub('([a-z0-9])([A-Z])', r'\1_\2', name).lower()

def toKebabCaseFromCamel(name):
    name = re.sub('(.)([A-Z][a-z]+)', r'\1-\2', name)
    return re.sub('([a-z0-9])([A-Z])', r'\1-\2', name).lower()


In [112]:
from pathlib import Path
import platform
import codecs

isWindows = platform.system() == 'Windows'

def writeFileData(filePath: Path, content):
    if isWindows:
        data = content
        with codecs.open(filePath, 'w', encoding= 'utf-8') as file:
            file.write(data)
    else:
        data = content
        with filePath.open('w') as file:
            file.write(data)

def addLinesToFileData(filePath: Path, content):
    if isWindows:
        data = content
        with codecs.open(filePath, 'a', encoding= 'utf-8') as file:
            file.write(data)
    else:
        data = content
        with filePath.open('a') as file:
            file.write(data)

def getFileData(filePath: Path):
    if isWindows:
        with codecs.open(filePath, 'r', 'utf-8') as file:
            content = file.read()
    else:
        with filePath.open() as file:
            content = file.read()
    return content

def replaceInFileData(filePath: Path, placeholder: str, newContent: str):
    fileContent = getFileData(filePath)
    fileContent = fileContent.replace(
        placeholder,
        newContent
    )
    writeFileData(filePath, fileContent)

In [113]:
sceneNameCapitaliseMask = "<#SceneNameCapitalise#>"
sceneNameCamelCaseMask = "<#SceneNameCamelCase#>"
fileContentTemplate = """import { logger } from 'src/app.logger'
import { UserService } from 'src/core/user/user.service'
import { Markup, Context } from 'telegraf'
import { Update } from 'telegraf/types'
import { SceneCallbackData } from '../models/scene-callback'
import { SceneEntrance } from '../models/scene-entrance.interface'
import { SceneName } from '../models/scene-name.enum'
import { SceneHandlerCompletion } from '../models/scene.interface'
import { Scene } from '../models/scene.abstract'
import { SceneUsagePermissionsValidator } from '../models/scene-usage-permissions-validator'
import { InjectableSceneConstructor } from '../scene-factory/scene-injections-provider.service'

// =====================
// Scene data classes
// =====================
export class <#SceneNameCapitalise#>SceneEntranceDto implements SceneEntrance.Dto {
    readonly sceneName = '<#SceneNameCamelCase#>'
}
type SceneEnterDataType = <#SceneNameCapitalise#>SceneEntranceDto
interface ISceneData {}

// =====================
// Scene main class
// =====================

@InjectableSceneConstructor()
export class <#SceneNameCapitalise#>Scene extends Scene<ISceneData, SceneEnterDataType> {
    // =====================
    // Properties
    // =====================

    readonly name: SceneName.union = '<#SceneNameCamelCase#>'
    protected get dataDefault(): ISceneData {
        return {} as ISceneData
    }
    protected get permissionsValidator(): SceneUsagePermissionsValidator.IPermissionsValidator {
        return new SceneUsagePermissionsValidator.CanUseIfNotBanned()
    }

    constructor(protected readonly userService: UserService) {
        super()
    }

    // =====================
    // Public methods
    // =====================

    async handleEnterScene(
        ctx: Context,
        data?: SceneEnterDataType
    ): Promise<SceneHandlerCompletion> {
        logger.log(
            `${this.name} scene handleEnterScene. User: ${this.user.telegramInfo.id} ${this.user.telegramInfo.username}`
        )
        await this.logToUserHistory(this.historyEvent.startScene<#SceneNameCapitalise#>)

        await ctx.replyWithHTML('Hello from <#SceneNameCapitalise#>', super.keyboardMarkupWithAutoLayoutFor(['Hello!']))

        return this.completion.inProgress({})
    }

    async handleMessage(ctx: Context, dataRaw: object): Promise<SceneHandlerCompletion> {
        logger.log(
            `${this.name} scene handleMessage. User: ${this.user.telegramInfo.id} ${this.user.telegramInfo.username}`
        )
        const message = ctx.message
        if (!message || !('text' in message)) return this.completion.canNotHandle({})

        await ctx.replyWithHTML(`Echo:\n${message.text}`)
        await ctx.replyWithHTML(`Goodbye`, Markup.removeKeyboard())

        return this.completion.complete()
    }

    async handleCallback(
        ctx: Context<Update.CallbackQueryUpdate>,
        data: SceneCallbackData
    ): Promise<SceneHandlerCompletion> {
        throw Error('Method not implemented.')
    }

    // =====================
    // Private methods
    // =====================
}

"""

## Configuration

In [114]:
sceneNameCapitalise = newSceneName[0].upper() + newSceneName[1:]
sceneNameCamelCase = newSceneName[0].lower() + newSceneName[1:]
sceneNameKebabCase = toKebabCaseFromCamel(sceneNameCapitalise)

In [115]:
# Check folder path

scenesPath = Path("../src/presentation/scenes/implementations/")
creatingSceneFolder = scenesPath
creatingSceneSourceFile = creatingSceneFolder / f"{sceneNameKebabCase}.scene.ts"

userHistoryEventFilePath = Path("../src/core/user/enums/user-history-event.enum.ts")
sceneNameFilePath = Path("../src/presentation/scenes/models/scene-name.enum.ts")
sceneFactoryFilePath = Path("../src/presentation/scenes/scene-factory/scene-factory.service.ts")
sceneEntranceInterfacePath = Path("../src/presentation/scenes/models/scene-entrance.interface.ts")


In [116]:
# Source code names update

fileContent = fileContentTemplate.replace(sceneNameCapitaliseMask, sceneNameCapitalise)
fileContent = fileContent.replace(sceneNameCamelCaseMask, sceneNameCamelCase)

## Source code generation

In [117]:
# creatingSceneFolder.mkdir(parents=True, exist_ok=True)
writeFileData(creatingSceneSourceFile, fileContent)

In [118]:
# UserHistoryEvent
placeholder = '/** New scene event placeholder */'
replaceInFileData(
    filePath=userHistoryEventFilePath,
    placeholder=placeholder,
    newContent=f"startScene{sceneNameCapitalise} = '{userHistoryEventSceneStart}',\n{tab}{placeholder}"
)

# SceneName
placeholder = '/** New scene name placeholder */'
replaceInFileData(
    filePath=sceneNameFilePath,
    placeholder=placeholder,
    newContent=f"'{sceneNameCamelCase}',\n{tab * 2}{placeholder}"
)

# SceneFactory
placeholder = '/** New scene import placeholder */'
replaceInFileData(
    filePath=sceneFactoryFilePath,
    placeholder=placeholder,
    newContent="import { " + sceneNameCapitalise + "Scene } from 'src/presentation/scenes/implementations/" +
    sceneNameKebabCase + ".scene'\n" + placeholder
)

placeholder = '/** New scene generation placeholder */'
replaceInFileData(
    filePath=sceneFactoryFilePath,
    placeholder=placeholder,
    newContent=f"case '{sceneNameCamelCase}':\n{tab * 4}return this.injectionsProvider.resolve({sceneNameCapitalise}Scene)\n{tab * 3}{placeholder}"
)

placeholder = '/** New scene entrance dto import placeholder */'
replaceInFileData(
	filePath=sceneEntranceInterfacePath,
	placeholder=placeholder,
    newContent="import { " + sceneNameCapitalise + "SceneEntranceDto } from 'src/presentation/scenes/implementations/" +
    sceneNameKebabCase + ".scene'\n" + placeholder
)

placeholder = '/** New scene entrance dto placeholder */'
replaceInFileData(
	filePath=sceneEntranceInterfacePath,
	placeholder=placeholder,
	newContent=f"{tab}| {sceneNameCapitalise}SceneEntranceDto\n{tab}{placeholder}"
)